# HOMEWORK 2 - PROGRAMMING PORTION
#### Genevieve Mortensen 2023-03-20
----------------------------------------

The purpose of this script is to compare cancer type predictions produced by Logistic Regression, Naive Bayes, and LDA classification methods based on gene mutation data.

The data is obtained here: https://www.cbioportal.org/study/summary?id=brca_tcga_pub2015

## Q2.1) Extract data

Here, we need to subset the mutation data for patients with invasive ductal and lobular carcinoma, which are the two most common types of breast cancer, respectively. This is the case where a patient's cancer (carcinoma) begins in the breast's milk transport duct or production gland (lobule) and then spreads to other tissues in the body.

                Based on the biological context, what does this mean for our classifier?

We want to classify the mutations characterizing these *two* types of cancer based on their distributions in the data. We therefore will create a __*binary classifier*__ using the three different methods aforementioned.

### Import Libraries

There are a LOT of libraries that we will need for this work.

In [125]:
#Import libraries

#Basic data handling libraries
import pandas as pd 
import numpy as np
import os
import matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline
sns.set(rc={'axes.facecolor':'lightblue'})

#Scikitlearn libraries
import sklearn
from sklearn import metrics
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.model_selection import StratifiedKFold, train_test_split, RepeatedStratifiedKFold, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.impute import KNNImputer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


# Statistics import
from statistics import mean, stdev


# Importing statsmodels
import statsmodels.api as sm

# Importing 'variance_inflation_factor'
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Importing warnings
# ------------------
import warnings
warnings.filterwarnings('ignore')

# Removing the limit for displaying rows and columns
# --------------------------------------------------
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

### Wrangle Data

The TA gave us most of the solution to this question, so we will use it here. There are many ways to pull out the relevant data, but basically we just need to have an sparse __n x d__ matrix, with our categories for classification also included in the dataframe.

In [126]:
## TA PROVIDED SOLUTION ##
# Reading "clinicalsample" dataset
df_clinicalsample = pd.read_csv(r"./brca_tcga_pub2015/brca_tcga_pub2015/data_clinical_sample.txt", sep="\t", skiprows=4)
df_clinicalsample.replace('[Not Available]', np.nan, inplace=True)
# df_clinicalpatient['Tumor_Sample_Barcode'] = df_clinicalpatient['PATIENT_ID']

# Reading "data_mutations.txt" dataset
df_mutations = pd.read_csv(r"./brca_tcga_pub2015/brca_tcga_pub2015/data_mutations.txt", sep="\t")

# Slicing Tumor Barcodes
df_mutations['PATIENT_ID'] = df_mutations['Tumor_Sample_Barcode'].apply(lambda x: x[:-3])


# Merging the values
# ------------------
df = pd.merge(df_mutations, df_clinicalsample, on='PATIENT_ID', how='left')
df.head()

# Filtering Out the important columns in the Dataframe, that we are going to use
# ------------------------------------------------------------------------------
cancer = df[['Hugo_Symbol','PATIENT_ID','Variant_Classification',
             'Variant_Type','Transcript_ID','Feature','Gene',
             'CANCER_TYPE','CANCER_TYPE_DETAILED',]]


# Filtering out Patients with the below 2 types of Carcinoma's
# --> Breast Invasive Ductal Carcinoma - "Infiltrating Ductal Carcinoma"
# --> Breast Invasice Lobular Carcinoma - "Infiltrating Lobular Carcinoma"
#
cancer = cancer.loc[(cancer['CANCER_TYPE_DETAILED']=='Breast Invasive Lobular Carcinoma') | (cancer['CANCER_TYPE_DETAILED']=='Breast Invasive Ductal Carcinoma')]

# Target Variable:
df_target = cancer[['PATIENT_ID','CANCER_TYPE_DETAILED']].drop_duplicates()

print(cancer.shape)
display(cancer)


(56933, 9)


,Hugo_Symbol,PATIENT_ID,Variant_Classification,Variant_Type,Transcript_ID,Feature,Gene,CANCER_TYPE,CANCER_TYPE_DETAILED
0,PTGER3,TCGA-B6-A0IG,Missense_Mutation,SNP,ENST00000356595,ENST00000356595,ENSG00000050628,Invasive Breast Carcinoma,Breast Invasive Ductal Carcinoma
1,FLG,TCGA-B6-A0IG,Missense_Mutation,SNP,ENST00000368799,ENST00000368799,ENSG00000143631,Invasive Breast Carcinoma,Breast Invasive Ductal Carcinoma
2,GPR52,TCGA-B6-A0IG,Silent,SNP,ENST00000367685,ENST00000367685,ENSG00000203737,Invasive Breast Carcinoma,Breast Invasive Ductal Carcinoma
3,SLC35F3,TCGA-B6-A0IG,Silent,SNP,ENST00000366618,ENST00000366618,ENSG00000183780,Invasive Breast Carcinoma,Breast Invasive Ductal Carcinoma
4,OR2T3,TCGA-B6-A0IG,Missense_Mutation,SNP,ENST00000359594,ENST00000359594,ENSG00000196539,Invasive Breast Carcinoma,Breast Invasive Ductal Carcinoma
...,...,...,...,...,...,...,...,...,...
62015,CCDC97,TCGA-BH-A42T,Missense_Mutation,SNP,ENST00000269967,ENST00000269967,ENSG00000142039,Invasive Breast Carcinoma,Breast Invasive Lobular Carcinoma
62016,SMG9,TCGA-BH-A42T,Missense_Mutation,SNP,ENST00000270066,ENST00000270066,ENSG00000105771,Invasive Breast Carcinoma,Breast Invasive Lobular Carcinoma
62017,ZNF324,TCGA-BH-A42T,Missense_Mutation,SNP,ENST00000536459,ENST00000536459,ENSG00000083812,Invasive Breast Carcinoma,Breast Invasive Lobular Carcinoma
62018,PLTP,TCGA-BH-A42T,Missense_Mutation,SNP,ENST00000477313,ENST00000477313,ENSG00000100979,Invasive Breast Carcinoma,Breast Invasive Lobular Carcinoma


Basically, the hard part was done for us. Thanks, Nitanshu!

### Preprocessing

Let's move on to creating the matrix. We can use One Hot Encoding to turn our mutations into binary categorical variables. This will give us a sparse matrix correlating to the number of times the mutation appears in a given patient. 
We will also need to retain the cancer type details for our binary classification.

In [127]:
#Get encoding for mutations
dummies = pd.get_dummies(cancer["Hugo_Symbol"], drop_first=True)
#concatenate cancer type
dummies = pd.concat([dummies, cancer["CANCER_TYPE_DETAILED"]], axis=1)
display(dummies)

,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,AADACL4,AADAT,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,CANCER_TYPE_DETAILED
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Lobular Carcinoma
62016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Lobular Carcinoma
62017,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Lobular Carcinoma
62018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Lobular Carcinoma


Now, let's fix our matrix such that our mutations are grouped by the patients and we retain cancer type details.

In [128]:
#concatenate full data frame for patient IDs
df = pd.concat([cancer, dummies], axis = 1)
#Group sparse matrix by patient IDs
sparse_df = df.groupby(["PATIENT_ID"])[dummies.columns].max()
sparse_df = sparse_df.loc[:,~sparse_df.columns.duplicated()]
display(sparse_df)


,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,AADACL4,AADAT,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,CANCER_TYPE_DETAILED
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-A1-A0SD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
TCGA-A1-A0SE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Lobular Carcinoma
TCGA-A1-A0SF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
TCGA-A1-A0SI,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
TCGA-A1-A0SJ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-LL-A5YP,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
TCGA-LQ-A4E4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Lobular Carcinoma
TCGA-MS-A51U,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Lobular Carcinoma


                    Cool! Now what?

Now we have our sparse matrix. But, we need to prepare our data some more for our model training.

In [129]:
# Fix up the dataframe
df_logir = sparse_df.copy()
df_logir.reset_index(level=0,inplace=True)
display(df_logir)

,PATIENT_ID,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,AADACL4,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,CANCER_TYPE_DETAILED
0,TCGA-A1-A0SD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
1,TCGA-A1-A0SE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Lobular Carcinoma
2,TCGA-A1-A0SF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
3,TCGA-A1-A0SI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
4,TCGA-A1-A0SJ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,TCGA-LL-A5YP,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
613,TCGA-LQ-A4E4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Lobular Carcinoma
614,TCGA-MS-A51U,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Lobular Carcinoma
615,TCGA-OL-A66I,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma


                  ~ Marvelous ~ Can we start classifying yet?

No! Let's drop off these categorical data so that we can define our training data, which must be numeric as our independent variable, and then our ground truth labels, or our dependent variables.

In [130]:
#Independent variable x
x = df_logir.drop('CANCER_TYPE_DETAILED', axis = 1)
x = x.drop('PATIENT_ID',axis=1)
#Dependent variable y
y = df_logir['CANCER_TYPE_DETAILED']
print("X and y shape are: ", x.shape, y.shape)


X and y shape are:  (617, 15301) (617,)


Great, now we have a matrix X which houses our data and a vector Y which houses our ground truth labels. Given some vector of data in X we will get one label in Y. In other words, given some mutations for a single patient, we will get one class of cancer.

## 2.2) Cross-validate classification models

Our classification models will be trained on our pre-processed data and then we will use them to make some prediction based on that training. First we will establish some variables which will be used throughout the model testing and then we will decipher the specifics of each method. For all methods testing, we are doing supervised learning.

In this step we scale the data to a normal distribution - that is, we transform and fit the data to a Gaussian distribution centered around a 0 mean.

In [131]:
#Scale input features
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

Next we establish a some scoring parameters. 
- Accuracy measures the total correctness of a model. It is the number of correct predictions a model made throughout the entire dataset. This doesn't work if the data is not class-balanced, meaning we have the same number of labels of each class in our training data.
- Precision measures how many labels for a class were correct. It is the number of correct labels for a class divided by the total number of labels assigned for a single class.
- Recall measures how many samples for a class label were correctly labeled. It is the number of correct labels for a single class divided by the sum of the labels given for one class which were correct and the other class which were incorrect.
- F1 score is the harmonic mean of precision and recall. It is the number of correct labels for a class divided by the sum of the correct labels for a class and half of the total number of incorrect labels for either class. This gives a better description of the quality of the classifier.

For all of these, the better classifier is one where the value of the mteric is closer to 1. However, precision and recall tend to fluctuate harmonically between them, and accuracy wouldn't be reliable since we don't know if our data is class-balanced. 

Given this, we will use F1 score to evaluate our data. Thankfully, we don't need to calculate this on our own, but we could easily do this given our precision and recall scores.

In [132]:
#scoring scheme for our model evaluation metrics
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

Next we need to split our data into k-folds where k = 5, since we are performing 5-fold cross-validation.

In [133]:
# Defining the Stratified K-Fold Cross Validation Object 
skf = KFold(n_splits=5, shuffle=True, random_state=42)

Awesome, we are done with the preparation. Now we can move on to classifying.


                                 Classify me, Captain!

### __Logistic Regression__

Logistic regression classification is where we define some threshold on a logistic curve and fit our data to it.
The logistic curve gives us a threshold at the inflection point and extrema at the asymptotes. For binary classification, this will work decently well- if a datum is above the threhold, it is given one class, below it, and it is given another.

In [134]:
# Defining the Logistic Regression Model Object
logir_ovr = LogisticRegression(random_state=0, 
                               multi_class='ovr')

In [135]:
# Define lists to hold Model Evaluation Metrics
logit_accuracies_1 = []
logit_precision_1 = []
logit_recall_1 = []
logit_f1scores_1 = []

For each index in our data, we have a training set and a testing set. We first establish the indeces for each iteration of our data, or fold, which is 5-fold. 4 of the folds are used to train and one is used to test. Then, we fit our model from our training data, which essentially just changes the logistic function so that it characterizes our data. Then, we predict a label using the testing fold. Lastly, we calculate the different metrics which tell us how well the model predicted a label using the testing fold of the data.

In the next iteration, we test on a different fold of the data, and train with the other remaining 4 folds. After 5 iterations of this, we have trained our model on all 5 folds of the data and tested it on all 5 folds.
The 'macro' parameter in our scoring function schemes averages the class-wise F1 scores, so this is how we obtain an F1 score for each iteration. We will store all of these.

In [136]:
# Perform 5-fold cross validation
for train_index, test_index in skf.split(x, y):
    
    #assign current training data from scaled independent variable and dependent variable
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    
    # Fit the model from our training data
    logir_ovr.fit(x_train_fold, y_train_fold)
    
    # use the model to predict the class based on 
    y_test_fold_pred = logir_ovr.predict(x_test_fold)
    
    #Store results
    logit_accuracies_1.append(logir_ovr.score(x_test_fold, y_test_fold))
    logit_f1scores_1.append(metrics.f1_score(y_test_fold, y_test_fold_pred, average='macro'))
    logit_precision_1.append(metrics.precision_score(y_test_fold, y_test_fold_pred, average='macro'))
    logit_recall_1.append(metrics.recall_score(y_test_fold, y_test_fold_pred, average='macro'))

Just to be thorough, we show the precision and recall metrics as well.

In [137]:
print(logit_precision_1)
print(logit_recall_1)
print(logit_f1scores_1)

[0.3951612903225806, 0.39919354838709675, 0.3983739837398374, 0.4146341463414634, 0.3780487804878049]
[0.5, 0.5, 0.5, 0.5, 0.5]
[0.44144144144144143, 0.4439461883408072, 0.4434389140271493, 0.45333333333333337, 0.4305555555555556]


### __Naive Bayes__

Naive Bayes classifier assumes strong independence between samples and computes conditional probability of a label given the training data. It is known to be a very good classifier because each feature contributes independently to the probability of data belonging to a class. The posterior probability of class given the data is equivalent to the probability of that class appearing in the data multiplied by the probability of the data belonging to a class divided by the probability of that data existing. This will give us probabilities for each class and the higher probability wins, thus we assign the data the corresponding class label, following Bayes Theorem. The primary advantage is that class boundaries are variable, not just linear. We would expect this classifier to be the best one.


In [138]:
# Define the object for Naive Bayes - Bernoulli NB is for boolean values, perfect for binary classification
naive_bayes = BernoulliNB()

#Create lists to hold metrics
naive_accuracies_1 = []
naive_precision_1 = []
naive_recall_1 = []
naive_f1scores_1 = []

In [139]:
for train_index, test_index in skf.split(x, y):  # 5 Times
    
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    
    naive_bayes.fit(x_train_fold, y_train_fold)
    
    # Predicting from predictor varialbes based on the model.
    y_test_fold_pred = naive_bayes.predict(x_test_fold)
    
    #Store results
    naive_accuracies_1.append(naive_bayes.score(x_test_fold, y_test_fold))
    naive_f1scores_1.append(metrics.f1_score(y_test_fold, y_test_fold_pred, average='macro'))
    naive_precision_1.append(metrics.precision_score(y_test_fold, y_test_fold_pred, average='macro'))
    naive_recall_1.append(metrics.recall_score(y_test_fold, y_test_fold_pred, average='macro'))
    

In [140]:
print(naive_precision_1)
print(naive_recall_1)
print(naive_f1scores_1)

[0.5649717514124294, 0.6033613445378152, 0.510233918128655, 0.47554347826086957, 0.5031512605042017]
[0.5180533751962323, 0.5248484848484849, 0.5042857142857143, 0.48949579831932777, 0.5005376344086022]
[0.4976851851851851, 0.5070336391437309, 0.48806881243063266, 0.47227077705386944, 0.4539400665926748]


### __LDA__

This dimension-reducing technique uses Bayes Theorem to assign classes in a similar way, but draws linear boundaries between data instead of variable boundaries, so our boundaries depend on the total probability of a set of data corresponding to a class. It is shorter to evaluate and is expected to perform better than logistic regression, but not as well as naive bayes.

In [141]:
# Define the object for LDA
lda = LinearDiscriminantAnalysis()

# Define the object K-Fold Cross Validation
lkfold = KFold(n_splits = 5,shuffle=True, random_state=42)

#Create lists to hold metrics
lda_accuracies_1 = []
lda_precision_1 = []
lda_recall_1 = []
lda_f1scores_1 = []

In [142]:
for train_index, test_index in skf.split(x, y):  # 5 Times
    
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    
    lda.fit(x_train_fold, y_train_fold)
    
    # Predicting from predictor varialbes based on the model.
    y_test_fold_pred = lda.predict(x_test_fold)
    
    #store results
    lda_accuracies_1.append(lda.score(x_test_fold, y_test_fold))
    lda_f1scores_1.append(metrics.f1_score(y_test_fold, y_test_fold_pred, average='macro'))
    lda_precision_1.append(metrics.precision_score(y_test_fold, y_test_fold_pred, average='macro'))
    lda_recall_1.append(metrics.recall_score(y_test_fold, y_test_fold_pred, average='macro'))
    

In [143]:
print(lda_precision_1)
print(lda_recall_1)
print(lda_f1scores_1)

[0.3951612903225806, 0.49357405140758875, 0.5060700169395822, 0.5207336523125997, 0.5552244039270687]
[0.5, 0.49151515151515157, 0.5087755102040816, 0.5364145658263306, 0.5564516129032258]
[0.44144144144144143, 0.3721312553429342, 0.4138071895424837, 0.47435897435897434, 0.414595452141724]


### Comparison of F1 Scores

The closer the score is to 1, the better the classifier.

In [144]:
def average(n):
    sum = 0
    for i in n:
        sum = sum + i         
    return sum / len(n)

print("Logistic Regression average F1 score is ", average(logit_f1scores_1))
print("Naive Bayes average F1 score is ", average(naive_f1scores_1))
print("LDA average F1 score is ", average(lda_f1scores_1))

Logistic Regression average F1 score is  0.44254308653965735
Naive Bayes average F1 score is  0.4837996960812186
LDA average F1 score is  0.4232668625655115


                Naive Bayes scored the highest. Yeehaw!